In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\w10\AppData\Local\Temp\ipykernel_4284\1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\w10\AppData\Local\Temp\ipykernel_4284\1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
spacex_df= pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [3]:
spacex_df.head(3)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0


In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
# Calculate other useful data.
payload_mark_values = range(0, 11000, 1000)
payload_mark_dict = dict(zip(payload_mark_values, payload_mark_values))

launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_site_options = [{'label': 'All Sites','value': 'ALL'}, *[{'label': site,'value': site} for site in launch_sites]]

In [6]:
# Create a dash application
app = dash.Dash(__name__)

In [7]:
# Create an app layout
app.layout = html.Div(
    children=[
        html.H1(
            'SpaceX Launch Records Dashboard',
            style={
                'textAlign': 'center',
                'color': '#503D36',
                'font-size': 40,
            }
        ),

        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites
        # dcc.Dropdown(id='site-dropdown',...)
        dcc.Dropdown(
            id='site-dropdown',
            options=launch_site_options,
            value='ALL',
            placeholder="Select a launch site here",
            searchable=True
        ),
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(
            dcc.Graph(
                id='success-pie-chart'
            )
        ),
        html.Br(),

        # TASK 3: Add a slider to select payload range
        html.P(
            "Payload range (Kg):"
        ),
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=1000,
            value=[0,10000],
            marks=payload_mark_dict
        ),

        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(
            dcc.Graph(
                id='success-payload-scatter-chart'
            )
        ),
    ]
)

In [8]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(
    Output(
        component_id='success-pie-chart',
        component_property='figure'
    ),
    Input(
        component_id='site-dropdown',
        component_property='value'
    )
)
def get_pie_chart(selected_site):
    if selected_site == 'ALL':
        data = spacex_df[spacex_df['class'] == 1]
        names = 'Launch Site'
        title='Total successful launches by site'
    else:
        data = spacex_df[spacex_df['Launch Site'] == selected_site]
        names = 'class'
        title=f'Total succesful launches for site {selected_site}'
    return px.pie(data, names=names, title=title)

In [9]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(
        component_id='success-payload-scatter-chart',
        component_property='figure'
    ),
    [
        Input(
            component_id='site-dropdown',
            component_property='value'
        ),
        Input(
            component_id="payload-slider",
            component_property="value"
        )
    ]
)
def get_scatter_chart(selected_site, selected_payload_range):
    low, high = selected_payload_range
    payload_mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if selected_site == 'ALL':
        filtered_df = spacex_df[payload_mask]
    else:
        filtered_df = spacex_df[payload_mask]
        filtered_df = filtered_df.loc[filtered_df['Launch Site'] == selected_site]
    return px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class', 
            color='Booster Version',
            hover_data=['Payload Mass (kg)']
        )

In [ ]:
# Run the app
# if __name__ == '__main__': # If file, but running from notebook, here.
app.run_server(dev_tools_silence_routes_logging=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off
